In [1]:
import segmentation_models as sm

import tensorflow as tf

sm.set_framework('tf.keras')

from cnn.input import PascalVOC2012Dataset, Dataloader, get_training_augmentation, get_preprocessing

import os

2022-07-10 18:59:52.238042: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-10 18:59:52.238057: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


KeyboardInterrupt: 

In [ ]:
train_dataset_descriptor_filepath = os.path.join(
    'pascalvoc12',
    'VOCdevkit',
    'VOC2012',
    'ImageSets',
    'Segmentation',
    'train.txt',
)

val_dataset_descriptor_filepath = os.path.join(
    'pascalvoc12',
    'VOCdevkit',
    'VOC2012',
    'ImageSets',
    'Segmentation',
    'val.txt',
)

images_path = os.path.join(
    'pascalvoc12', 
    'VOCdevkit', 
    'VOC2012', 
    'JPEGImages'
)

masks_path = os.path.join(
    'pascalvoc12', 
    'VOCdevkit', 
    'VOC2012', 
    'SegmentationClass'
) 

In [ ]:
image_height = 224
image_width = 224
backbone = 'efficientnetb0'

In [ ]:
train_dataset = PascalVOC2012Dataset(
    train_dataset_descriptor_filepath,
    images_path=images_path,
    masks_path=masks_path,
    image_height=image_height,
    image_width=image_width,
    #augmentation=get_training_augmentation(image_height, image_width),
    preprocessing=get_preprocessing(sm.get_preprocessing(backbone))
)

val_dataset = PascalVOC2012Dataset(
    val_dataset_descriptor_filepath,
    images_path=images_path,
    masks_path=masks_path,
    image_height=image_height,
    image_width=image_width,
    preprocessing=get_preprocessing(sm.get_preprocessing(backbone))
)

/home/guilherme/git/segqnas/.venv/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2029: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  warnings.warn('Using lambda is incompatible with multiprocessing. '


In [ ]:
train_dataloader = Dataloader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = Dataloader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
net = sm.FPN(backbone, classes=21, input_shape=(image_height, image_width, 3), activation='softmax', encoder_weights='imagenet')

2022-07-10 18:45:08.319961: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 18:45:08.320141: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guilherme/git/segqnas/.venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-10 18:45:08.320786: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guilherme/git/segqnas/.venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-10 18:45:08.321024: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcub

In [ ]:
net.compile(                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    optimizer='Adam',
    loss=sm.losses.categorical_focal_dice_loss,
    metrics=[sm.metrics.IOUScore(threshold=0.5)],
)

In [ ]:
history = net.fit(train_dataloader, 
                validation_data=val_dataloader,
                epochs=50,
                callbacks=[
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss", mode="min", verbose=1, patience=5
                    )
                ],
            )

Epoch 1/50
